In [41]:
import pandas as pd
# gensim
from gensim import corpora, models, similarities, matutils
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
import pyLDAvis.gensim
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
df = pd.read_json('~/Metis/projects/04-fletcher/.gitignore/files/projects.json')

In [5]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(df.description)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
counts = count_vectorizer.transform(df.description).transpose()
print(counts.shape)

(1514396, 6342)


In [8]:
corpus = matutils.Sparse2Corpus(counts)

In [46]:
corpus_vect_gensim = gensim.matutils.Sparse2Corpus(counts, documents_columns=False)

# transform scikit vocabulary into gensim dictionary
vocabulary_gensim = {}
for key, val in count_vectorizer.vocabulary_.items():
    vocabulary_gensim[val] = key

In [9]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [36]:
corpora.Dictionary(count_vectorizer.vocabulary_.items())

2017-08-18 13:39:44,085 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: invalid continuation byte

In [10]:
list(count_vectorizer.vocabulary_.items())[0:10]

[('known', 738020),
 ('meant', 837432),
 ('help', 617753),
 ('reiki', 1111784),
 ('life', 768775),
 ('changed', 199954),
 ('better', 125691),
 ('way', 1458663),
 ('body', 139690),
 ('energy', 423592)]

In [ ]:
%time lda = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, passes=10)

2017-08-18 11:33:29,923 : INFO : using symmetric alpha at 0.1
2017-08-18 11:33:29,924 : INFO : using symmetric eta at 6.60329266585e-07
2017-08-18 11:33:30,173 : INFO : using serial LDA version on this node
2017-08-18 11:36:17,294 : INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 6342 documents, updating every 14000 documents, evaluating every ~6342 documents, iterating 50x with a convergence threshold of 0.001000
2017-08-18 11:36:17,297 : INFO : training LDA model using 7 processes
2017-08-18 11:36:17,664 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/6342, outstanding queue size 1


In [49]:
pyLDAvis.gensim.prepare(topic_model=lda, corpus=corpus, dictionary=vocabulary_gensim)

AttributeError: 'dict' object has no attribute 'token2id'

In [50]:
vocabulary_gensim.token2id()

AttributeError: 'dict' object has no attribute 'token2id'

In [51]:
dictionary = corpora.Dictionary(df.description)

2017-08-18 15:55:30,594 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [32]:
dictionary = corpora.Dictionary(texts)

NameError: name 'texts' is not defined

In [42]:
nmf_model = NMF(2)
dtm_nmf = nmf_model.fit_transform(counts)
dtm_nmf = Normalizer(copy=False).fit_transform(counts)

/home/brendanfitzpatrick/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)
